# Adapting-CLIP

In [12]:
######## Mount the drive ########
from google.colab import drive
drive.mount('/content/drive/')

######## DIRPATH ########
DIRPATH_GRIT = '/content/drive/MyDrive/grit_official/'
DIRPATH_DATA = os.path.join(DIRPATH_GRIT, 'data/downloaded_logs/default_job/data/downloaded/GRIT/')
DIRPATH_LOCAL = '/content/GRIT-LOCAL/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
######## Install the dependencies ########
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-k3r8ypiq
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-k3r8ypiq
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369398 sha256=0a897e131b73abb2f55115ed350f467f81ec8f56d16e3411ad59af27d6ad15b7
  Stored in directory: /tmp/pip-ephem-wheel-cache-61hap9xj/wheels/c8/e4/e1/11374c111387672fc2068dfbe0d4b424cb9cdd1b2e184a71b5
Successfully built clip


In [18]:
import os
import json
from PIL import Image

## Load images from Google Drive 

In [7]:
# Load GRIT data
if(not os.path.exists(DIRPATH_LOCAL)):
    os.makedirs(DIRPATH_LOCAL)
    %cd /content/
    !cp /content/drive/MyDrive/COLAB-FILES/12lbs/grit_local.tar.gz /content/
    !tar -xzf grit_local.tar.gz    

/content


In [9]:
%cd /content/drive/MyDrive/adapting-CLIP/

#import argparse
#import os.path as osp
from tqdm import tqdm
import numpy as np
import torch
from models.slic_vit import SLICViT
from models.ss_baseline import SSBaseline
from models.resnet_high_res import ResNetHighRes
from utils.zsg_data import FlickrDataset, VGDataset
from utils.grounding_evaluator import GroundingEvaluator

/content/drive/MyDrive/adapting-CLIP


In [ ]:
# ! python eval.py --model vit14 --dataset flickr_s1_val --iou_thr 0.5 --num_samples 500

model = SLICViT
args = {
    'model': 'vit14',
    'alpha': 0.75,
    'aggregation': 'mean',
    'n_segments': list(range(100, 601, 50)),
    'temperature': 0.02,
    'upsample': 2,
    'start_block': 0,
    'compactness': 50,
    'sigma': 0,
}
dataset_full = FlickrDataset(data_type='flickr30k_c1/val')
model = model(**args).cuda()
#model.eval()

## Load json file

In [15]:
# Chekc the images in the json file
with open(os.path.join(DIRPATH_DATA,'samples/ablation/localization.json')) as f:
    data = json.load(f)

    # Example of the json element
    # [
    #     {
    #         "example_id": "coco_loc_test-reserve_spoon_527067",
    #         "image_id": "coco/test2015/COCO_test2015_000000527067.jpg",
    #         "output_options": null,
    #         "task_bbox": null,
    #         "task_name": "localization",
    #         "task_query": "spoon"
    #     },
    #     ... 
    # ]

In [16]:
# Check the number of the images
print(len(data))

# Check if the images are in the folder using vectorization
def checkfile(di):
  return os.path.exists(os.path.join(DIRPATH_LOCAL,'images',di['image_id']))
cnt = len(list(filter(checkfile,data)))
print("ALL THE IMAGES EXISTING - ",cnt==len(data),cnt,len(data))

# Check the types of the images need to be downloaded
folderset=set({})
for di in data:
  folderset.add("/".join(di['image_id'].split('/')[:-1]))
  # check if the image exists
folderset

21078
ALL THE IMAGES EXISTING -  True 21078 21078


{'coco/test2015',
 'distorted/localization/coco/test2015',
 'nyuv2',
 'open_images/test'}

## Evaluate the model on the GRIT-Bench dataset

In [ ]:
## Predict
output = []
# Output format example
# [
#    {
#        "example_id" : str
#        "confidence" : float in [0,1]
#        "bboxes"     : 2d list of int [[x1,y1,x2,y2],...] # box coordinates, per instance
#    },
#    ...
#]
for json_chunk in tqdm(data):
    image_path = os.path.join(DIRPATH_LOCAL,'images',json_chunk['image_id'])
    if(not os.path.exists(image_path)):
        print("Image not found - ",image_path)
        break

    # Load the image
    image = np.array(Image.open(image_path).convert("RGB"))
    # Load the text
    text = json_chunk['task_query']
    
    # Predict
    pred_boxes, _ = model(image, text)

    # Append the result
    output_chunk = {
        "example_id" : json_chunk['example_id'],
        "confidence" : 0.5, #float(pred_scores.mean()),
        "bboxes"     : pred_boxes.tolist()
    }
    output.append(output_chunk)

    # Save the intermediate results
    if(len(output)%500==0):
        OUTPUT_FILENAME = f'/content/drive/MyDrive/COLAB-FILES/12lbs/jsonfiles/ablation_localization_{len(output)//500}.json'
        with open(OUTPUT_FILENAME, 'w') as f:
            json.dump(output, f)

  1%|          | 108/21078 [06:52<22:09:33,  3.80s/it]

In [ ]:
OUTPUT_FILENAME = '/content/drive/MyDrive/COLAB-FILES/12lbs/localization.json'
with open(OUTPUT_FILENAME, 'w') as f:
    json.dump(output, f)

In [25]:
# Count the number of the pparameters(need for the GRIT submission)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total Parameters: {total_params}")
print(f"Trainable Parameters: {trainable_params}")

Total Parameters: 428402945
Trainable Parameters: 428402945
